In [15]:
import pandas as pd
import re
from datetime import datetime

In [16]:
file_path = 'payment_16-Apr-2025.xls'

In [17]:
df = pd.read_excel(file_path)
#df.head()

In [18]:
# Select specific columns
df1 = df[['UID', 'BudgetItem', 'Department', 'Amount', 'Vendor/Transfer to department', 'AmountRemarks']]
#df1.head()

In [19]:
# Rename specific columns
df2 = df1.rename(columns={'UID': 'Entry No', 'BudgetItem': 'Dr Ledger Name', 'Department':'Dr Cost Center', 'Amount':'Dr Amt', 'Vendor/Transfer to department':'Cr Ledger Name' })
#df2.head()

In [20]:
# Modified function to extract Bill No., Invoice No., or Quotation Ref No.
def extract_reference(text):
    bill_match = re.search(r'Bill No\.\s*(\S+)', text)
    invoice_match = re.search(r'Invoice No\.\s*(\S+)', text)
    quotation_match = re.search(r'Quotation Ref No\.\s*(\S+)', text)
    
    if bill_match:
        return bill_match.group(1)
    elif invoice_match:
        return invoice_match.group(1)
    elif quotation_match:
        return quotation_match.group(1)
    
    return None

# Apply the function to your data frame
df2['Bill Ref No.'] = df2['AmountRemarks'].apply(extract_reference)

# Display the first 50 rows
#df2.head()

In [21]:
# Create the new 'Vch Narration' column by merging 'Entry No.' and 'AmountRemarks'
df2['Vch Narration'] = 'UID No. ' + df2['Entry No'].astype(str) + ' ' + df2['AmountRemarks']
# Display the updated dataframe
#df2.head()

In [22]:
df3=df2[['Entry No', 'Dr Ledger Name',	'Dr Cost Center', 'Dr Amt', 'Cr Ledger Name', 'Bill Ref No.', 'Vch Narration' ]]
#df3.head()

In [23]:
# Adding blank columns at specific positions
df3.insert(0, 'Date', '')  # Add 'Date' at the first position
df3.insert(2, 'Vch Name', '')  # Add 'Vch No' at the third position
df3.insert(7, 'Cr Cost Center', '')  # Add 'cost center' at the eighth position
df3.insert(8, 'Cr Amt', '')  # Add 'cost center' at the ninth position
# Display the updated dataframe
df3.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4250000881,,TRA- Field Travel & Transportation Cost,RDD-REP-AZTECH,14421,Sai Service Private Limited,,,177/BC/25000072,UID No. 4250000881 Vehicle Servicing and Maint...
1,,4250000877,,ADMN-Administrative Expenses,RDD-REP-AZTECH,130,MSEDCL-003464,,,000002815538064,UID No. 4250000877 Electricity Bill Charges fo...
2,,4250000876,,ADMN-Administrative Expenses,RDD-REP-AZTECH,130,MSEDCL-003464,,,000002815538062,UID No. 4250000876 Electricity Bill Charges fo...
3,,4250000875,,ADMN-Administrative Expenses,RDD-REP-AZTECH,200,MSEDCL-003464,,,000002815538298,UID No. 4250000875 Electricity Bill Charges fo...
4,,4250000872,,PRM-Food Expenses,RDD-REP-Vidyavan (Gauranga Prabhu),340,Mahalaxmi Cloth Store Pro Jain,,,074,UID No. 4250000872 Purchases of Carry Bags for...


In [24]:
df4 = df3[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name','Cr Amt','Cr Cost Center','Vch Narration','Bill Ref No.']]
df4.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
0,,4250000881,,TRA- Field Travel & Transportation Cost,14421,RDD-REP-AZTECH,Sai Service Private Limited,,,UID No. 4250000881 Vehicle Servicing and Maint...,177/BC/25000072
1,,4250000877,,ADMN-Administrative Expenses,130,RDD-REP-AZTECH,MSEDCL-003464,,,UID No. 4250000877 Electricity Bill Charges fo...,000002815538064
2,,4250000876,,ADMN-Administrative Expenses,130,RDD-REP-AZTECH,MSEDCL-003464,,,UID No. 4250000876 Electricity Bill Charges fo...,000002815538062
3,,4250000875,,ADMN-Administrative Expenses,200,RDD-REP-AZTECH,MSEDCL-003464,,,UID No. 4250000875 Electricity Bill Charges fo...,000002815538298
4,,4250000872,,PRM-Food Expenses,340,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,,,UID No. 4250000872 Purchases of Carry Bags for...,074


In [25]:
# Ensure 'Dr Amt' is a string and remove commas
df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
# Convert 'Dr Amt' to numeric
df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])
# Create a copy of df3 to df4
df4 = df3.copy()
# Calculate cumulative sum for each 'Entry No'
df4['Cr Amt'] = df3.groupby('Entry No')['Dr Amt'].transform('sum')
# Format 'Dr Amt' and 'Cr Amt' with commas and two decimal places
df4['Dr Amt'] = df4['Dr Amt'].apply(lambda x: f"{x:,.2f}")
df4['Cr Amt'] = df4['Cr Amt'].apply(lambda x: f"{x:,.2f}")
# Display the final DataFrame
df4.head(5)

C:\Users\grd00\AppData\Local\Temp\ipykernel_14512\2643644666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
C:\Users\grd00\AppData\Local\Temp\ipykernel_14512\2643644666.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])


,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4250000881,,TRA- Field Travel & Transportation Cost,RDD-REP-AZTECH,"14,421.00",Sai Service Private Limited,,"14,421.00",177/BC/25000072,UID No. 4250000881 Vehicle Servicing and Maint...
1,,4250000877,,ADMN-Administrative Expenses,RDD-REP-AZTECH,130.00,MSEDCL-003464,,130.00,000002815538064,UID No. 4250000877 Electricity Bill Charges fo...
2,,4250000876,,ADMN-Administrative Expenses,RDD-REP-AZTECH,130.00,MSEDCL-003464,,130.00,000002815538062,UID No. 4250000876 Electricity Bill Charges fo...
3,,4250000875,,ADMN-Administrative Expenses,RDD-REP-AZTECH,200.00,MSEDCL-003464,,200.00,000002815538298,UID No. 4250000875 Electricity Bill Charges fo...
4,,4250000872,,PRM-Food Expenses,RDD-REP-Vidyavan (Gauranga Prabhu),340.00,Mahalaxmi Cloth Store Pro Jain,,340.00,074,UID No. 4250000872 Purchases of Carry Bags for...


In [26]:
df4 = df4[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name', 'Cr Amt', 'Cr Cost Center', 'Vch Narration','Bill Ref No.']]
from datetime import datetime

# Fill 'Date' column with today's date in DD/MM/YYYY format
df4['Date'] = datetime.today().strftime('%d/%m/%Y')

# Fill 'Vch Name' column with "Journal"
df4['Vch Name'] = 'Journal'

df4.tail()

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
1,16/04/2025,4250000877,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000877 Electricity Bill Charges fo...,000002815538064
2,16/04/2025,4250000876,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000876 Electricity Bill Charges fo...,000002815538062
3,16/04/2025,4250000875,Journal,ADMN-Administrative Expenses,200.00,RDD-REP-AZTECH,MSEDCL-003464,200.00,,UID No. 4250000875 Electricity Bill Charges fo...,000002815538298
4,16/04/2025,4250000872,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000872 Purchases of Carry Bags for...,074
5,16/04/2025,4250000871,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000871 Purchases of Carry Bags for...,086


In [27]:
df5 = df4.drop_duplicates()
df5.tail()

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
1,16/04/2025,4250000877,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000877 Electricity Bill Charges fo...,000002815538064
2,16/04/2025,4250000876,Journal,ADMN-Administrative Expenses,130.00,RDD-REP-AZTECH,MSEDCL-003464,130.00,,UID No. 4250000876 Electricity Bill Charges fo...,000002815538062
3,16/04/2025,4250000875,Journal,ADMN-Administrative Expenses,200.00,RDD-REP-AZTECH,MSEDCL-003464,200.00,,UID No. 4250000875 Electricity Bill Charges fo...,000002815538298
4,16/04/2025,4250000872,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000872 Purchases of Carry Bags for...,074
5,16/04/2025,4250000871,Journal,PRM-Food Expenses,340.00,RDD-REP-Vidyavan (Gauranga Prabhu),Mahalaxmi Cloth Store Pro Jain,340.00,,UID No. 4250000871 Purchases of Carry Bags for...,086


In [28]:
# Get current date and timestamp
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# Create output file name with date and timestamp
output_filename = f'ICM_to_TDL_Sheet_{current_time}.xlsx'
# Convert DataFrame to Excel
df5.to_excel(output_filename, index=False)